In [6]:
import pandas as pd
import random
import openpyxl

In [7]:
# ====================================================================================#
# Configurações iniciais
# ====================================================================================#
# Semente para reprodutibilidade (42 e 81)
random.seed(42)



# Variáveis de interesse
# Estrato 1: Centros (Sites)
estrato_centros = [18,19,20,21,22,23,24,25,26,27]  # Lista de centros (códigos dos centros)

#Meta do total de participantes por centro
#Cade centro tem uma meta unica de participantes, mas o total de participantes no estudo (somando todos os centros) é fixo em total_participantes
meta_participantes_por_centro = {
    18: 12,
    19: 12,
    20: 12,
    21: 12,
    22: 12,
    23: 12,
    24: 12,
    25: 12,
    26: 12,
    27: 12
}

#tamanho da amostra (n)
total_participantes = sum(meta_participantes_por_centro.values())


# os participentes serão alocados em cada centro de acordo com a meta de participantes por centro
numero_centros = len(estrato_centros)
# Verificar se a soma das metas de participantes por centro é igual ao total de participantes
assert sum(meta_participantes_por_centro.values()) == total_participantes, (
    f"A soma das metas de participantes por centro ({sum(meta_participantes_por_centro.values())})"
    f" deve ser igual ao total de participantes ({total_participantes})."
)

# Distribuição de participantes por centro
participantes_por_centro = total_participantes // numero_centros

# Estrato 2: Sexo
estrato_genero = ['Masculino', 'Feminino']


# ====================================================================================#
# Parâmetros de randomização em blocos (block randomization)
# ====================================================================================#
## Tamanho do bloco
tamanho_bloco = 4
bracos = ['2506091', '2506092']  # Braços do estudo
blocos_por_centro = participantes_por_centro // tamanho_bloco

# Distribuição de participantes por genero por centro
# Cada Centro deverá ter 50% de participantes masculinos e 50% de participantes femininos
# Cada centro tem uma meta distinta de participantes (meta_participantes_por_centro), mas a distribuição de gênero é fixa
# Portanto, o número de participantes por gênero em cada centro será metade do total de participantes por centro
participantes_por_genero_por_centro = participantes_por_centro // 2
participantes_por_genero_por_centro_por_braco = participantes_por_genero_por_centro // 2
print(participantes_por_centro)
print(participantes_por_genero_por_centro)
print(numero_centros)
print(blocos_por_centro)

12
6
10
3


In [8]:
# ====================================================================================#
# Distribuição aleatória de ampolas por centro
# ====================================================================================#
## Ampolas do grupo 2506091 (numeradas de 1 a 108)

etiquetas_2506091 = list(range(1, 109))  # Lista de ampolas do grupo 2506091 (numeradas de 1 a 108)
random.shuffle(etiquetas_2506091)  # Embaralha as ampolas para garantir aleatoriedade

## Ampolas do grupo 2506092 (numeradas de 109 a 216)
etiquetas_2506092 = list(range(109, 217))  # Lista de ampolas do grupo 2506092 (numeradas de 109 a 216)
random.shuffle(etiquetas_2506092)  # Embaralha as ampolas para garantir aleatoriedade




In [9]:
import pandas as pd

def distribuir_por_braco(centro: int,
                         braco: str,
                         meta: int,
                         etiquetas: list[int]
                        ) -> list[tuple]:
    """
    Para um dado centro e braço:
     - meta: total de participantes neste centro
     - assume 50% participantes por braço (meta//2)
     - dentro de cada braço, 50% homens, 50% mulheres
     - cada homem leva 2 etiquetas, cada mulher 1 etiqueta
     - retorna lista de tuplas (centro, genero, braco, etiquetas_str)
     - consome etiquetas da lista passada (etiquetas.pop(0))
    """
    linhas = []
    n_por_braco = meta // 2
    n_por_genero = n_por_braco // 2  # ex: meta=12 → n_por_braco=6 → n_por_genero=3

    # alocar homens: 2 etiquetas cada
    for _ in range(n_por_genero):
        dois = [etiquetas.pop(0) for _ in range(2)]
        etiquetas_str = "/ ".join(f"{x:03}" for x in dois)
        linhas.append((centro, "Masculino", braco, etiquetas_str))

    # alocar mulheres: 1 etiqueta cada
    for _ in range(n_por_genero):
        uma = etiquetas.pop(0)
        linhas.append((centro, "Feminino", braco, f"{uma:03}"))

    return linhas

# --- usa as listas globais já embaralhadas:
resultado = []
for centro in estrato_centros:
    meta = meta_participantes_por_centro[centro]
    # 2506091
    resultado += distribuir_por_braco(centro, "2506091", meta, etiquetas_2506091)
    # 2506092
    resultado += distribuir_por_braco(centro, "2506092", meta, etiquetas_2506092)

# monta o DataFrame final
df_final = pd.DataFrame(
    resultado,
    columns=["Centro", "Genero", "Braco", "Etiquetas"]
)


In [10]:
# ====================================================================================#
# Exportar os resultados para csv e excel
# ====================================================================================#
# Exportar para CSV na pasta csv
df_final.to_csv(r'..\csv\randomizacao_imox_semente42.csv', index=False, encoding='utf-8-sig')

# Exportar para Excel na pasta excel
with pd.ExcelWriter(r'..\xlsx\randomizacao_imox_semente42.xlsx', engine='openpyxl') as writer:
    df_final.to_excel(writer, index=False, sheet_name='Randomizacao Imox Semente 42')
    # Formatação da planilha
    workbook = writer.book
    worksheet = writer.sheets['Randomizacao Imox Semente 42']
    for col in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in col)
        adjusted_width = (max_length + 2)
        worksheet.column_dimensions[col[0].column_letter].width = adjusted_width
